<a href="https://colab.research.google.com/github/lutakrystal305/Reinforcement_Learning/blob/main/QLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym
!pip install keras-rl2

     |████████████████████████████████| 61kB 6.0MB/s 


In [ ]:
import gym
import random

env = gym.make('MountainCar-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n
print('observation: ', states)
print(env.observation_space)
print('actions: ', actions)

observation:  2
Box(-1.2000000476837158, 0.6000000238418579, (2,), float32)
actions:  3


In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    #state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

AssertionError: ignored

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy

In [ ]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(10, states)))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dense(actions, activation='linear'))
    return model


In [ ]:
model = build_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                672       
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 51        
Total params: 1,251
Trainable params: 1,251
Non-trainable params: 0
____________________________________________________

In [ ]:
def build_agent(model, actions):
    policy = EpsGreedyQPolicy()
    memory = SequentialMemory(limit=50000, window_length=10)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn


In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 19:10 - reward: -1.0000

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 143s 14ms/step - reward: -1.0000
51 episodes - episode_reward: -194.196 [-200.000, -146.000] - loss: 1.365 - mae: 19.052 - mean_q: -28.104

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 146s 15ms/step - reward: -1.0000
58 episodes - episode_reward: -172.603 [-200.000, -105.000] - loss: 2.053 - mae: 33.395 - mean_q: -49.269

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 141s 14ms/step - reward: -1.0000
65 episodes - episode_reward: -154.723 [-200.000, -92.000] - loss: 1.455 - mae: 33.679 - mean_q: -49.690

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 144s 14ms/step - reward: -1.0000
63 episodes - episode_reward: -156.381 [-200.000, -86.000] - loss: 1.102 - mae: 31.615 - mean_q: -46.754

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 152s 15ms/step - reward: -1.0000
done, took 725.509 seconds


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)


In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
print(dqn.policy)

In [ ]:
import numpy as np
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: -90.000, steps: 90
Episode 2: reward: -122.000, steps: 122
Episode 3: reward: -161.000, steps: 161
Episode 4: reward: -200.000, steps: 200
Episode 5: reward: -152.000, steps: 152
Episode 6: reward: -176.000, steps: 176
Episode 7: reward: -200.000, steps: 200
Episode 8: reward: -170.000, steps: 170
Episode 9: reward: -175.000, steps: 175
Episode 10: reward: -153.000, steps: 153
-159.9


In [ ]:
!pip install pyvirtualdisplay

In [ ]:
!apt install xvfb -y
!pip install pyglet

from IPython import display
import matplotlib.pyplot as plt
import pyvirtualdisplay

_display = Display(visible=False, size=(1400, 900))
_ = _display.start()

img = plt.imshow(env.render('rgb_array'))

_ = dqn.test(env, nb_episodes=15, visualize=True)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.9).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


GLException: ignored

In [ ]:
!pip install 'imageio==2.4.0'

In [ ]:
pip install pyglet==1.5.11

     |████████████████████████████████| 1.1MB 8.6MB/s 
ERROR: gym 0.17.3 has requirement pyglet<=1.5.0,>=1.4.0, but you'll have pyglet 1.5.11 which is incompatible.
  Found existing installation: pyglet 1.5.0
    Uninstalling pyglet-1.5.0:
      Successfully uninstalled pyglet-1.5.0


In [ ]:
import imageio
from IPython.display import HTML
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)

    return HTML("""
    <video width="640" height="480" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """.format(b64.decode()))

def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
    filename = filename + ".mp4"
    with imageio.get_writer(filename, fps=fps) as video:
        for _ in range(num_episodes):
            time_step = env.reset()
            video.append_data(env.render())
            while not time_step.is_last():
                action_step = policy.action(time_step)
                time_step = eval_env.step(action_step.action)
                video.append_data(eval_py_env.render())
    return embed_mp4(filename)




create_policy_eval_video(dqn.policy, "trained-agent")

NoSuchDisplayException: ignored

In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

env.reset()
for _ in range(100):
    plt.imshow(env.render(mode='rgb_array'))
    display.display(plt.gcf())
    display.clear_output(wait=True)
    action = env.action_space.sample()
    env.step(action)

NoSuchDisplayException: ignored